### Batch registration of morphologies
This notebook consumes a csv file called 'morphology_registration.csv' The columns are name, filename, and mtype. The filename should be without an extension. The script looks for morphologies files with the .h5, .asc, and .swc extensions. An example file is as follows:

name	filename	mtype
test1	file1	L5_PC
test2	file2	L5_PC

The user should modify this notebook to make sure the common species, brain_region, brain_location (optional), agent, and role are specified. The notebook will then read in the csv file and register the morphologies.


In [75]:
import io
import os
import tempfile
from pathlib import Path

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models import (
    BrainLocation,
    BrainRegion,
    Contribution,
    MTypeClass,
    MTypeClassification,
    Organization,
    ReconstructionMorphology,
    Role,
    Species,
    Strain,
)

In [76]:
# Example usage:
# Create a dummy CSV file for demonstration
#with open('morphology_registration.csv', 'w', newline='', encoding='utf-8') as f:
#    f.write("name,filename\n")
#    f.write("morphology1,file1.swc,L5_PC\n")
#    f.write("morphology2,file2.swc,L5_PC\n")
#create dummy payloads for testing
#file1 = Path("."", "file1.h5")
#file1.write_text("h5")
#file2 = Path("."", "file2.swc")
#file2.write_text("swc")

### Initialize the entitycore client

In [77]:
#if running entitycore locally
#entitycore_api_url = "http://0.0.0.0:8000" 
#project_context = ProjectContext(
#    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
#    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
#)
#token = os.getenv("ACCESS_TOKEN", "XXX")
#client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)


# uncomment for staging
from obi_auth import get_token
token = get_token(environment="staging")
# Replace this with your vlab project url in staging
project_context = ProjectContext.from_vlab_url("https://staging.openbraininstitute.org/app/virtual-lab/lab/bf7d398c-b812-408a-a2ee-098f633f7798/project/100a9a8a-5229-4f3d-aef3-6a4184c59e74/home")
client = Client(environment="staging", project_context=project_context, token_manager=token)


## Search for entities

In [78]:
species = client.search_entity(entity_type=Species, query={"name": "Mus musculus"}, limit=10).one()

In [79]:
rprint(species)

Species(
    id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 183929, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 183929, tzinfo=TzInfo(UTC)),
    name='Mus musculus',
    taxonomy_id='NCBITaxon:10090'
)

In [80]:
strain = client.search_entity(entity_type=Strain, query={"name": "Cux2-CreERT2"}).one()

In [81]:
rprint(strain)

Strain(
    id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 56, 853949, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 56, 853949, tzinfo=TzInfo(UTC)),
    name='Cux2-CreERT2',
    taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
    species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
)

In [82]:
brain_region = client.search_entity(entity_type=BrainRegion, query={"annotation_value": 68}).one()

In [83]:
rprint(brain_region)

BrainRegion(
    id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
    name='Frontal pole, layer 1',
    annotation_value=68,
    acronym='FRP1',
    parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
    hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
    color_hex_triplet='268F45'
)

### Create a location object

In [84]:
brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)

## Register morphologies

In [85]:
import csv
import os

def read_and_process_csv(file_path):
    """
    Reads a CSV file, processes each filename, and checks for the existence
    of files with .h5, .swc, and .asc extensions.

    Args:
        file_path (str): The path to the CSV file.
    """
    idlist = []
    #see if morphologies are available
    try:
        with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)

            # Skip the header row
            next(csv_reader)

            # Iterate over the remaining rows
            for row in csv_reader:
                if len(row) >= 3:
                    mtype_label = row[2]
                    try:
                        mtype = client.search_entity(entity_type=MTypeClass, query={"pref_label": mtype_label}).one()
                    except Exception as e:
                        print(f"An error occurred when searching for the mtype: {mtype_label}")
                        print("Registration aborted. Please fix the mtype in the csv file and try again.")
                        raise e
                    
                    #if mtype is None:
                    #    print(f"Warning: MTypeClass with label '{mtype_label}' not found.")
                    
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")

    #get an agent and role for the contribution
    #here we assume first organization and role
    #but you should register your agent and role and then retrieve it here
    #for registering the contribution in the loop below
    agent = client.search_entity(entity_type=Organization, limit=1).one()
    role = client.search_entity(entity_type=Role, limit=1).one()
   

    try:
        with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)

            # Skip the header row
            next(csv_reader)

            # Iterate over the remaining rows
            for row in csv_reader:
                if len(row) >= 2:
                    name = row[0]
                    original_filename = row[1]
                    
                    print(f"Processing Name: {name}, Original Filename: {original_filename}")

                    #make a morphology object
                    morphology = ReconstructionMorphology(
                        name=name,
                        description="A morphology",
                        species=species,
                        strain=strain,
                        brain_region=brain_region,
                        location=brain_location,
                        legacy_id=None,
                        authorized_public=False,
                    )
                    #register the morphology
                    registered = client.register_entity(entity=morphology)
                    idlist.append(registered.id)

                    if len(row) >= 3:
                        mtype_label = row[2]
                        
                        mtype = client.search_entity(entity_type=MTypeClass, query={"pref_label": mtype_label}).one()
                        try:
                            mtype_classification = MTypeClassification(
                               mtype_class_id=mtype.id, entity_id=registered.id, authorized_public=True
                            )
                            mtype_classification = client.register_entity(mtype_classification)
                        except Exception as e:
                            print(f"An error occurred when registering the mtype: {mtype_label} {e}")
                            raise e

                    # Strip the original filename extension
                    base_filename, _ = os.path.splitext(original_filename)

                    # Check for .h5 file
                    h5_filename = base_filename + ".h5"
                    if os.path.exists(h5_filename):
                        print(f"  Found .h5 file: {h5_filename}")
                        # use a filepath to register first asset
                        asset1 = client.upload_file(
                            entity_id=registered.id,
                            entity_type=ReconstructionMorphology,
                            file_path=h5_filename,
                            file_content_type="application/x-hdf5",
                            asset_label="morphology",
                            
                        )
                        #rprint(asset1)

                    # Check for .swc file
                    swc_filename = base_filename + ".swc"
                    if os.path.exists(swc_filename):
                        print(f"  Found .swc file: {swc_filename}")
                        # use a filepath to register asset
                        asset1 = client.upload_file(
                            entity_id=registered.id,
                            entity_type=ReconstructionMorphology,
                            file_path=swc_filename,
                            file_content_type="application/swc",
                            asset_label="morphology",
                            
                        )
                        #rprint(asset1)

                    # Check for .asc file
                    asc_filename = base_filename + ".asc"
                    if os.path.exists(asc_filename):
                        print(f"  Found .asc file: {asc_filename}")
                        # use a filepath to register asset
                        asset1 = client.upload_file(
                            entity_id=registered.id,
                            entity_type=ReconstructionMorphology,
                            file_path=asc_filename,
                            file_content_type="application/asc",
                            asset_label="morphology",
                            
                        )
                        #rprint(asset1)
                    contribution = Contribution(
                        agent=agent,
                        role=role,
                        entity=registered,
                        )    
                    registered_contribution = client.register_entity(entity=contribution)

    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return idlist



# Call the function to read the csv file
idlist=read_and_process_csv('morphology_registration.csv')

Processing Name: test1, Original Filename: file1.swc
  Found .swc file: file1.swc
Processing Name: test2, Original Filename: file2.swc
  Found .swc file: file2.swc


In [86]:
rprint(idlist)

[UUID('285ccc35-5e8b-4b69-adeb-61e0f99fdf15'), UUID('f921a651-e48d-4d87-9b09-468692ac9e8b')]

## Retrieve the registered morphologies

In [87]:
# with assets and mtypes
for id in idlist:
    fetched = client.get_entity(entity_id=id, entity_type=ReconstructionMorphology)
    rprint(fetched)

ReconstructionMorphology(
    assets=[
        Asset(
            id=UUID('81a3c3b2-cff4-4304-9587-f1c095e10783'),
            update_date=None,
            creation_date=None,
            path='file1.swc',
            full_path='private/bf7d398c-b812-408a-a2ee-098f633f7798/100a9a8a-5229-4f3d-aef3-6a4184c59e74/assets/rec
onstruction_morphology/285ccc35-5e8b-4b69-adeb-61e0f99fdf15/file1.swc',
            is_directory=False,
            content_type=<ContentType.application_swc: 'application/swc'>,
            size=314849,
            sha256_digest='26f5874c552643e47b2057f3f3ac29b3882297a570b420f7ac278554d1e43de5',
            status=<AssetStatus.created: 'created'>,
            meta={},
            label=<AssetLabel.morphology: 'morphology'>
        )
    ],
    id=UUID('285ccc35-5e8b-4b69-adeb-61e0f99fdf15'),
    update_date=datetime.datetime(2025, 8, 28, 7, 57, 57, 161898, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 8, 28, 7, 57, 57, 161898, tzinfo=TzInfo(UTC)),
    name='test1',
    description='A morphology',
    type=<EntityType.reconstruction_morphology: 'reconstruction_morphology'>,
    created_by=Person(
        id=UUID('c51a9d79-39c5-45c6-b57b-9197e5d8c0cc'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Daniel Keller',
        given_name='Daniel',
        family_name='Keller'
    ),
    updated_by=Person(
        id=UUID('c51a9d79-39c5-45c6-b57b-9197e5d8c0cc'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Daniel Keller',
        given_name='Daniel',
        family_name='Keller'
    ),
    authorized_public=False,
    authorized_project_id=UUID('100a9a8a-5229-4f3d-aef3-6a4184c59e74'),
    contributions=[
        Contribution(
            id=UUID('cf7e33ea-2f47-4b51-b113-1228890de289'),
            update_date=None,
            creation_date=None,
            agent=Organization(
                id=UUID('a385509d-240b-43fd-8702-0d2c345cb1a7'),
                update_date=None,
                creation_date=None,
                type='organization',
                pref_label='University of Toronto, Toronto, Ontario, Canada',
                alternative_name=''
            ),
            role=Role(
                id=UUID('6a8018fd-790b-46e6-9ea6-dc994fa82228'),
                update_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 198426, tzinfo=TzInfo(UTC)),
                creation_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 198426, tzinfo=TzInfo(UTC)),
                name='Brain Atlas Pipeline Executor role',
                role_id='nsg:BrainAtlasPipelineExecutionRole'
            ),
            entity=None
        )
    ],
    legacy_id=None,
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    brain_region=BrainRegion(
        id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Frontal pole, layer 1',
        annotation_value=68,
        acronym='FRP1',
        parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='268F45'
    ),
    species=Species(
        id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
        update_date=None,
        creation_date=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
        update_date=None,
        creation_date=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
    ),
    license=None,
    mtypes=[
        MTypeClass(
            id=UUID('0c807abc-2124-43e

ReconstructionMorphology(
    assets=[
        Asset(
            id=UUID('96a28824-3c64-4040-b335-10f7761a15ba'),
            update_date=None,
            creation_date=None,
            path='file2.swc',
            full_path='private/bf7d398c-b812-408a-a2ee-098f633f7798/100a9a8a-5229-4f3d-aef3-6a4184c59e74/assets/rec
onstruction_morphology/f921a651-e48d-4d87-9b09-468692ac9e8b/file2.swc',
            is_directory=False,
            content_type=<ContentType.application_swc: 'application/swc'>,
            size=314849,
            sha256_digest='26f5874c552643e47b2057f3f3ac29b3882297a570b420f7ac278554d1e43de5',
            status=<AssetStatus.created: 'created'>,
            meta={},
            label=<AssetLabel.morphology: 'morphology'>
        )
    ],
    id=UUID('f921a651-e48d-4d87-9b09-468692ac9e8b'),
    update_date=datetime.datetime(2025, 8, 28, 7, 57, 58, 185842, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 8, 28, 7, 57, 58, 185842, tzinfo=TzInfo(UTC)),
    name='test2',
    description='A morphology',
    type=<EntityType.reconstruction_morphology: 'reconstruction_morphology'>,
    created_by=Person(
        id=UUID('c51a9d79-39c5-45c6-b57b-9197e5d8c0cc'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Daniel Keller',
        given_name='Daniel',
        family_name='Keller'
    ),
    updated_by=Person(
        id=UUID('c51a9d79-39c5-45c6-b57b-9197e5d8c0cc'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Daniel Keller',
        given_name='Daniel',
        family_name='Keller'
    ),
    authorized_public=False,
    authorized_project_id=UUID('100a9a8a-5229-4f3d-aef3-6a4184c59e74'),
    contributions=[
        Contribution(
            id=UUID('1123e89a-90f9-4982-80b5-d7032dfed1f9'),
            update_date=None,
            creation_date=None,
            agent=Organization(
                id=UUID('a385509d-240b-43fd-8702-0d2c345cb1a7'),
                update_date=None,
                creation_date=None,
                type='organization',
                pref_label='University of Toronto, Toronto, Ontario, Canada',
                alternative_name=''
            ),
            role=Role(
                id=UUID('6a8018fd-790b-46e6-9ea6-dc994fa82228'),
                update_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 198426, tzinfo=TzInfo(UTC)),
                creation_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 198426, tzinfo=TzInfo(UTC)),
                name='Brain Atlas Pipeline Executor role',
                role_id='nsg:BrainAtlasPipelineExecutionRole'
            ),
            entity=None
        )
    ],
    legacy_id=None,
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    brain_region=BrainRegion(
        id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Frontal pole, layer 1',
        annotation_value=68,
        acronym='FRP1',
        parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='268F45'
    ),
    species=Species(
        id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
        update_date=None,
        creation_date=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
        update_date=None,
        creation_date=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
    ),
    license=None,
    mtypes=[
        MTypeClass(
            id=UUID('0c807abc-2124-43e